In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle


import tensorflow as tf
import tensorflow
from tensorboard.plugins.histogram.summary import histogram
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime



data=pd.read_csv("Churn_Modelling.csv")
data.head()
#preprocessing the data
##droping irrelevant columns
data=data.drop(["RowNumber","CustomerId","Surname"],axis=1)
# print(data)

##encode categorical variable
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
# print(data)

##one hot encoding for geographical column
one_hot_geo=OneHotEncoder(sparse_output=False)
geo_encoder=one_hot_geo.fit_transform(data[['Geography']])
# print(geo_encoder)
geo_name=one_hot_geo.get_feature_names_out(['Geography'])
geo_encoded_df=pd.DataFrame(geo_encoder,columns=geo_name)

##combine new data
data=data.drop('Geography',axis=1)
data=pd.concat([data,geo_encoded_df],axis=1)
##print(data)


with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)


with open('one_hot_geo.pkl','wb') as file:
    pickle.dump(one_hot_geo,file)


"""deviding my data into dependent and independent dataset"""

X=data.drop('Exited',axis=1)
Y=data['Exited']

"""splitting the data into train and test"""
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,train_size=0.2,random_state=42)

"""scale this feature"""
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

# print(x_train)
# print(x_test)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

# print(x_train.shape[1])
"""build our ANN Model"""
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),       ##1st hidden layer connected with input layer
    Dense(32,activation='relu'), ##2nd hidden layer
    Dense(1,activation='sigmoid')##output layer
]

)

model.summary()

"""compile the model"""
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
print(loss)
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

"""setup the tensorboard"""

log_dir="logs/fit" + datetime.datetime.now().strftime("Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

"""set up early stopping"""
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

"""train the model"""
history=model.fit(
    X_train,Y_train,validation_data=(X_test,Y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

model.save('model.h5')

"""losd tensorboard extention"""



Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
63/63 [==============================] - 1s 9ms/step - loss: 0.4531 - accuracy: 0.8005 - val_loss: 0.4137 - val_accuracy: 0.8179
Epoch 2/100
63/63 [==============================] - 0s 7ms/step - loss: 0.3950 - accuracy: 0.8340 - val_loss: 0.4032 - val_accuracy: 0.8276
Ep

c:\Users\Hp\PycharmProjects\ANN-Regration\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


'losd tensorboard extention'